# Credit Risk Scorecard Demo

## Before you begin
To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also create a documentation project on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
# %pip install --upgrade validmind

## Initialize the client library
In a browser, go to the Client Integration page of your documentation project and click Copy to clipboard next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

This step requires a documentation project. Learn how you can create one.

Next, replace this placeholder with your own code snippet:

In [2]:
import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "2494c3838f48efe590d531bfe225d90b",
  api_secret = "4f692f8161f128414fef542cab2a4e74834c75d01b3a8e088a1834f2afcfe838",
  project = "clk00h0u800x9qjy67gduf5om"
)

2023-08-06 09:41:11,672 - INFO(validmind.api_client): Connected to ValidMind. Project: [6] Credit Risk Scorecard - Initial Validation (clk00h0u800x9qjy67gduf5om)


## Use case

### Setup

#### Introduction

The **Credit risk Scorecard** model created from the Lending Club dataset is instrumental in computing the Probability of Default (PD), a key factor in ECL calculations. This scorecard assesses several credit characteristics of potential borrowers, like their credit history, income, outstanding debts, and more, each of which is assigned a specific score. By combining these scores, we derive a total score for each borrower, which translates into an estimated Point-in-Time (PiT) PD. The PiT PD reflects the borrower's likelihood of default at a specific point in time, accounting for both current and foreseeable future conditions.

Additionally, for a holistic view of credit risk, it's essential to estimate the Lifetime PD. The Lifetime PD, as the name suggests, predicts the borrower's likelihood of default throughout the life of the exposure, taking into account potential future changes in the economic and financial conditions.

#### Import Libraries

In [3]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

# Standard library imports
import re
import pickle
from datetime import datetime
from typing import List

# Data handling and analysis imports
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Visualization imports
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# File handling import
import zipfile

# Scorecard development
import scorecardpy as sc


#### Processing Functions

In [4]:
unused_variables = [
    "id", "member_id", "funded_amnt", "emp_title", "url", "desc", "application_type",
    "title", "zip_code", "delinq_2yrs", "mths_since_last_delinq", "mths_since_last_record",
    "revol_bal", "total_rec_prncp", "total_rec_late_fee", "recoveries", "out_prncp_inv", "out_prncp", 
    "collection_recovery_fee", "next_pymnt_d", "initial_list_status", "pub_rec",
    "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "pymnt_plan",
    "tot_coll_amt", "tot_cur_bal", "total_rev_hi_lim", "last_pymnt_d", "last_credit_pull_d",
    'earliest_cr_line', 'issue_d']

In [5]:
def save_model_and_df(model, df, base_filename):
    """Save a model and a dataframe with a timestamp in the filename"""
    # Get current date and time
    now = datetime.now()

    # Convert the current date and time to string
    timestamp_str = now.strftime("%Y%m%d_%H%M%S")

    filename = f'{base_filename}_{timestamp_str}.pkl'

    # Save the model and dataframe
    with open(filename, 'wb') as file:
        pickle.dump((model, df), file)
        
    print(f"Model and dataframe saved as {filename}")

def get_numerical_columns(df):
        numerical_columns = df.select_dtypes(
            include=["int", "float", "uint"]
        ).columns.tolist()
        return numerical_columns

def get_categorical_columns(df):
        categorical_columns = df.select_dtypes(
            include=["object", "category"]
        ).columns.tolist()
        return categorical_columns

def add_target_column(df, target_column):
    # Assuming the column name is 'loan_status'
    df[target_column] = df['loan_status'].apply(lambda x: 0 if x == "Fully Paid" else 1 if x == "Charged Off" else np.nan)
    # Remove rows where the target column is NaN
    df = df.dropna(subset=[target_column])
    # Convert target column to integer
    df[target_column] = df[target_column].astype(int)
    return df

def variables_with_min_missing(df, min_missing_percentage):
    # Calculate the percentage of missing values in each column
    missing_percentages = df.isnull().mean() * 100

    # Get the variables where the percentage of missing values is greater than the specified minimum
    variables_to_drop = missing_percentages[missing_percentages > min_missing_percentage].index.tolist()

    # Also add any columns where all values are missing
    variables_to_drop.extend(df.columns[df.isnull().all()].tolist())

    # Remove duplicates (if any)
    variables_to_drop = list(set(variables_to_drop))

    return variables_to_drop

def clean_term_column(df, column):
    """
    Function to remove 'months' string from the 'term' column and convert it to categorical
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")
    
    df[column] = df[column].str.replace(' months', '')
    
    # Convert to categorical
    df[column] = df[column].astype('object')

def clean_emp_length_column(df, column):
    """
    Function to clean 'emp_length' column and convert it to categorical.
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")
    
    df[column] = df[column].replace('n/a', np.nan)
    df[column] = df[column].str.replace('< 1 year', str(0))
    df[column] = df[column].apply(lambda x: re.sub('\D', '', str(x)))
    df[column].fillna(value = 0, inplace=True)

    # Convert to categorical
    df[column] = df[column].astype('object')

def clean_inq_last_6mths(df, column):
    """
    Function to convert 'inq_last_6mths' column into categorical.
    """
    # Ensure the column exists in the dataframe
    if column not in df.columns:
        raise ValueError(f"The column '{column}' does not exist in the dataframe.")

    # Convert to categorical
    df[column] = df[column].astype('category')

def compute_outliers(series, threshold=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

def remove_iqr_outliers(df, target_column, threshold=1.5):
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols.remove(target_column)  # Exclude target_column from numerical columns
    for col in num_cols:
        outliers = compute_outliers(df[col], threshold)
        df = df[~df[col].isin(outliers)]
    return df

In [6]:
# 1. Import Raw Data
def import_raw_data(): 
    # Download the Lending Club dataset from a public S3 bucket
    lending_club_url = "https://vmai.s3.us-west-1.amazonaws.com/datasets/lending_club_loan_data_2007_2014.csv"
    df_out = pd.read_csv(lending_club_url)
    return df_out

# 2. Data Preparation: Add Definition of Default
def data_preparation_add_default_definition(df, default_column):
    
    df_out = df.copy()
    df_out = add_target_column(df_out, default_column)

    # Drop 'loan_status' variable 
    df_out.drop(columns='loan_status', axis=1, inplace=True)

    # Remove unused variables
    df_out = df_out.drop(columns=unused_variables)

    # Remove missing values
    min_missing_count = 80
    variables_to_drop = variables_with_min_missing(df_out, min_missing_count)
    df_out.drop(columns=variables_to_drop, axis=1, inplace=True)
    df_out.dropna(axis=0, subset=["emp_length"], inplace=True)
    df_out.dropna(axis=0, subset=["revol_util"], inplace=True)

    # Format variable types
    clean_emp_length_column(df_out, 'emp_length')
    clean_term_column(df_out, 'term')
    clean_inq_last_6mths(df_out, 'inq_last_6mths')

    # Remove outliers
    df_out = remove_iqr_outliers(df_out, default_column, threshold=1.5)

    return df_out

# 3. Data Sampling: Data Split 
def data_sampling(df, target_column):
    df_out = df.copy()

    # Split data into train and test 
    X = df_out.drop(target_column, axis = 1)
    y = df_out[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 42, stratify = y)

    # Concatenate X_train with y_train to form df_train
    df_train = pd.concat([X_train, y_train], axis=1)

    # Concatenate X_test with y_test to form df_test
    df_test = pd.concat([X_test, y_test], axis=1)

    return df_train, X_train, y_train, df_test, X_test, y_test

# 4. EDA: Drop Unused Categories
def eda_drop_categories(df):

    df_out = df.copy()
 
    # Select rows where purpose is 'debt_consolidation' or 'credit_card'
    df_out = df_out[df_out['purpose'].isin(['debt_consolidation', 'credit_card'])]
    
    # Remove rows where grade is 'F' or 'G'
    df_out = df_out[~df_out['grade'].isin(['F', 'G'])]

    # Remove rows where sub_grade starts with 'F' or 'G'
    df_out = df_out[~df_out['sub_grade'].str.startswith(('F', 'G'))]
    
    # Remove rows where home_ownership is 'OTHER', 'NONE', or 'ANY'
    df_out = df_out[~df_out['home_ownership'].isin(['OTHER', 'NONE', 'ANY'])]

    return df_out

# 5. EDA: Drop Unused Features
def drop_features(df, features_to_drop):
    df_out = df.copy()
    df_out = df_out.drop(columns = features_to_drop, axis=1)
    return df_out 

# 5. EDA: Convert Features to WoE Values
def convert_to_woe(df, woe_df, target_col):
    
    df_out = df.copy()
    
    # Create bins from woe_df
    bins = transform_woe_df(woe_df)
    
    # Make sure we don't transform the target column
    if target_col in bins:
        del bins[target_col]
    
    # Apply the WoE transformation
    df_out = sc.woebin_ply(df_out, bins=bins)
    
    return df_out

def transform_woe_df(woe_df):
    # Select and rename columns
    transformed_df = woe_df[['variable', 'bin', 'count', 'count_distr', 'good', 'bad', 'badprob', 'woe', 'bin_iv', 'total_iv']].copy()
    transformed_df.rename(columns={
        'bin_iv': 'total_iv'
    }, inplace=True)
    
    # Create 'is_special_values' column (assuming there are no special values)
    transformed_df['is_special_values'] = False
    
    # Transform 'bin' column into interval format and store it in 'breaks' column
    transformed_df['breaks'] = transformed_df['bin'].apply(lambda x: '[-inf, %s)' % x if isinstance(x, float) else '[%s, inf)' % x)
    
    # Group by 'variable' to create bins dictionary
    bins = {}
    for variable, group in transformed_df.groupby('variable'):
        bins[variable] = group
    
    return bins

# 6. Model Training: Add Constant
def add_constant(df, target_column):
    
    df_out = df.copy()
    
    # Add constant to X_train for intercept term
    y = df_out[target_column]
    X = df_out.drop(target_column, axis=1)
    X = sm.add_constant(X)
    df_out = pd.concat([X, y], axis=1)

    return df_out, X, y

## Data Description

#### Import Raw Data: Dataset 1

In [7]:
df_1_raw_data = import_raw_data()

/var/folders/tn/rbr74q892k13m82y37y396h40000gn/T/ipykernel_60171/3836106577.py:5: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_out = pd.read_csv(lending_club_url)


#### **Assess Dataset 1: raw data**

In [8]:
from validmind.vm_models.test_context import TestContext
from validmind.tests.data_validation.DescriptiveStatistics import DescriptiveStatistics

vm_df_1_raw_data = vm.init_dataset(dataset=df_1_raw_data)
test_context_1_raw_data = TestContext(dataset=vm_df_1_raw_data)

metric = DescriptiveStatistics(test_context_1_raw_data)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-06 09:41:50,297 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:41:50,297 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [9]:
from validmind.tests.data_validation.MissingValuesBarPlot import MissingValuesBarPlot

params = {"threshold": 70,
          "fig_height": 1100}

metric = MissingValuesBarPlot(test_context_1_raw_data, params)
metric.run()
await metric.result.log()
metric.result.show()

## Data Preparation

#### Add Definition of Default: Dataset 2

In [10]:
target_column = 'default'
df_2_default_definition = data_preparation_add_default_definition(
    df = df_1_raw_data, 
    default_column = target_column)

/var/folders/tn/rbr74q892k13m82y37y396h40000gn/T/ipykernel_60171/2227589160.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### **Assess Dataset 2: definition of default**

In [11]:
from validmind.tests.data_validation.ClassImbalance import ClassImbalance

vm_df_2_default_definition = vm.init_dataset(dataset=df_2_default_definition,
                        target_column=target_column)
test_context_2_default_definition = TestContext(dataset=vm_df_2_default_definition)

metric = ClassImbalance(test_context_2_default_definition)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-06 09:42:02,134 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:42:02,138 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [12]:
from validmind.tests.data_validation.IQROutliersTable import IQROutliersTable

num_features = get_numerical_columns(df_2_default_definition)
params = {"num_features": num_features,
          "threshold": 1.5
        }

metric = IQROutliersTable(test_context_2_default_definition, params)
metric.run()
await metric.result.log()
metric.result.show()

In [13]:
from validmind.tests.data_validation.IQROutliersBarPlot import IQROutliersBarPlot

num_features = get_numerical_columns(df_2_default_definition)
params = {"num_features": num_features,
          "threshold": 1.5,
          "fig_width": 500}

metric = IQROutliersBarPlot(test_context_2_default_definition, params)
metric.run()
await metric.result.log()
metric.result.show()

## Data Sampling

#### Sampling Method

We employ stratified sampling to create our training and testing sets. Stratified sampling is particularly important in this context. When the `stratify = y` parameter is set, it ensures that the distribution of the target variable (`y`) in the test set is the same as that in the original dataset. 

This is crucial for maintaining a consistent representation of the target variable classes, especially important in scenarios where the classes are imbalanced, which is often the case in credit risk scorecards.

#### Data Split: Train and Test from Dataset 2

In [14]:
df_train_2, X_train, y_train, df_test_2, X_test, y_test = data_sampling(
    df = df_2_default_definition,
    target_column=target_column)

## Exploratory Data Analysis 

#### **Assess Train Dataset 2**

In [15]:
from validmind.tests.data_validation.TabularNumericalHistograms import TabularNumericalHistograms

vm_df_train_2 = vm.init_dataset(dataset=df_train_2,
                                target_column=target_column)
test_context_train_2 = TestContext(dataset=vm_df_train_2)

metric = TabularNumericalHistograms(test_context_train_2)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-06 09:42:25,810 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:42:25,811 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [16]:
from validmind.tests.data_validation.HighCardinality import HighCardinality
metric = HighCardinality(test_context_train_2)
metric.run()
await metric.result.log()
metric.result.show()

In [17]:
from validmind.tests.data_validation.TabularCategoricalBarPlots import TabularCategoricalBarPlots
metric = TabularCategoricalBarPlots(test_context_train_2)
metric.run()
await metric.result.log()
metric.result.show()

#### Drop Categories: Train and Test Dataset 3

In [18]:
df_train_3_eda_drop_categories = eda_drop_categories(df_train_2)
df_test_3_eda_drop_categories = eda_drop_categories(df_test_2)

#### **Assess Train Dataset 3: drop categories**

In [19]:
from validmind.tests.data_validation.TargetRateBarPlots import TargetRateBarPlots

vm_df_train_3_eda_drop_categories = vm.init_dataset(
    dataset=df_train_3_eda_drop_categories, 
    target_column=target_column)

test_context_train_3_eda_drop_categories = TestContext(
    dataset=vm_df_train_3_eda_drop_categories)

# Configure the metric
params = {
    "default_column": target_column,
    "columns": None
}

metric = TargetRateBarPlots(test_context_train_3_eda_drop_categories, params=params)
metric.run()
await metric.result.log()
metric.result.show()

2023-08-06 09:43:18,651 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:43:18,652 - INFO(validmind.vm_models.dataset): Inferring dataset types...


The column default is correct and contains only 1 and 0.


#### Drop Features: Train and Test Dataset 4 

In [20]:
features_to_drop = ['addr_state', 'total_rec_int', 'loan_amnt',
                    'funded_amnt_inv', 'dti', 'revol_util', 'total_pymnt', 
                    'total_pymnt_inv', 'last_pymnt_amnt']

df_train_4_eda_drop_features = drop_features(
    df=df_train_3_eda_drop_categories, 
    features_to_drop = features_to_drop)

df_test_4_eda_drop_features = drop_features(
    df=df_test_3_eda_drop_categories, 
    features_to_drop = features_to_drop)

#### **Assess Train Dataset 4: drop features**

In [21]:
from validmind.tests.data_validation.ChiSquaredFeaturesTable import ChiSquaredFeaturesTable

vm_df_train_4_eda_drop_features = vm.init_dataset(dataset=df_train_4_eda_drop_features, target_column=target_column)
test_context_train_4_eda_drop_features = TestContext(dataset=vm_df_train_4_eda_drop_features)

cat_features = get_categorical_columns(df_train_4_eda_drop_features)
params = {"cat_features": cat_features,
          "p_threshold": 0.05}

metric = ChiSquaredFeaturesTable(test_context_train_4_eda_drop_features, params)
metric.run()
await metric.result.log() 
metric.result.show()

2023-08-06 09:43:29,679 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:43:29,681 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [22]:
from validmind.tests.data_validation.ANOVAOneWayTable import ANOVAOneWayTable

num_features = get_numerical_columns(df_train_4_eda_drop_features)
params = {"num_features": num_features,
          "p_threshold": 0.05}

metric = ANOVAOneWayTable(test_context_train_4_eda_drop_features, params)
metric.run()
await metric.result.log()
metric.result.show()

In [23]:
from validmind.tests.data_validation.PearsonCorrelationMatrix import PearsonCorrelationMatrix

params = {"declutter": False,
          "features": None,
          "fontsize": 13}

metric = PearsonCorrelationMatrix(test_context_train_4_eda_drop_features, params)
metric.run()
await metric.result.log()
metric.result.show()

In [24]:
from validmind.tests.data_validation.FeatureTargetCorrelationPlot import FeatureTargetCorrelationPlot

params = {"features": None}

metric = FeatureTargetCorrelationPlot(test_context_train_4_eda_drop_features, params)
metric.run()
await metric.result.log()
metric.result.show()

## Feature Engineering

#### **Assess Train Dataset 4: class binning and merging**

In [25]:
from validmind.tests.data_validation.WOEBinTable import WOEBinTable

# Run test
metric = WOEBinTable(test_context_train_4_eda_drop_features)
metric.run()
await metric.result.log()
woe_iv_dic = metric.result.metric.value['woe_iv']
metric.result.show()

Running with breaks_adj: None
Performing binning with breaks_adj: None
[INFO] creating woe binning ...


In [26]:
# Set test parameters
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

# Run test
metric = WOEBinTable(test_context_train_4_eda_drop_features, params)
metric.run()
await metric.result.log()
woe_iv_dic = metric.result.metric.value['woe_iv']
metric.result.show()

Running with breaks_adj: {'int_rate': [5, 10, 15]}
Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...


In [27]:
from validmind.tests.data_validation.WOEBinPlots import WOEBinPlots

# Set test parameters
params = {
    "breaks_adj": {"int_rate": [5,10,15]},
    "fig_height": 500,
}

# Run test
metric = WOEBinPlots(test_context_train_4_eda_drop_features, params=params)
metric.run()
await metric.result.log()
metric.result.show()

[INFO] creating woe binning ...


## Model Training

#### Convert Features into WoE Values: Train and Test Dataset 5

In [28]:
# Compute WoE 
params = {
    "breaks_adj": {
        "int_rate": [5,10,15]}  
     }

metric = WOEBinTable(test_context_train_4_eda_drop_features, params=params)
metric.run()
woe_dic = metric.result.metric.value['woe_iv']
woe_df = pd.DataFrame(woe_dic)

Running with breaks_adj: {'int_rate': [5, 10, 15]}
Performing binning with breaks_adj: {'int_rate': [5, 10, 15]}
[INFO] creating woe binning ...


In [29]:
df_train_5_convert_to_woe = convert_to_woe(
    df=df_train_4_eda_drop_features,
    woe_df=woe_df,
    target_col=target_column) 

df_test_5_convert_to_woe = convert_to_woe(
    df=df_test_4_eda_drop_features, 
    woe_df=woe_df, 
    target_col=target_column) 

[INFO] converting into woe values ...
[INFO] converting into woe values ...


#### Add Constant: Train and Test Dataset 6

In [30]:
df_train_6_add_const, X_train, y_train = add_constant(
    df=df_train_5_convert_to_woe,
    target_column=target_column)

df_test_6_add_const, X_test, y_test = add_constant(
    df=df_test_5_convert_to_woe,
    target_column=target_column)

#### Fit Model 1

In [31]:
# Define the model
model = sm.GLM(y_train, X_train, family=sm.families.Binomial())

# Fit the model
model_fit_1_candidate = model.fit()

# Print out the statistics
print(model_fit_1_candidate.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:               105815
Model:                            GLM   Df Residuals:                   105800
Model Family:                Binomial   Df Model:                           14
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -45535.
Date:                Sun, 06 Aug 2023   Deviance:                       91071.
Time:                        09:44:19   Pearson chi2:                 1.06e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.07964
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

#### Drop Features: Train and Test Dataset 7

In [32]:
features_to_drop = []

df_train_7_drop_features = drop_features(
    df=df_train_6_add_const, 
    features_to_drop=features_to_drop)

df_test_7_drop_features = drop_features(
    df=df_test_6_add_const, 
    features_to_drop=features_to_drop)

#### Fit Model 2 

In [33]:
# Create X_train and y_train
y_train = df_train_7_drop_features[target_column]
X_train = df_train_7_drop_features.drop(target_column, axis=1)
y_test = df_test_7_drop_features[target_column]
X_test = df_test_7_drop_features.drop(target_column, axis=1)

# Define the model
model = sm.GLM(y_train, X_train, family=sm.families.Binomial())

# Fit the model
model_fit_2_final = model.fit()

# Save the model and train dataset for PD development 
save_data = True
if save_data:
    save_model_and_df(model_fit_2_final, df=df_train_7_drop_features, base_filename='model_fit_glm_scorecard')

# Print out the statistics
print(model_fit_2_final.summary())

Model and dataframe saved as model_fit_glm_scorecard_20230806_094419.pkl
                 Generalized Linear Model Regression Results                  
Dep. Variable:                default   No. Observations:               105815
Model:                            GLM   Df Residuals:                   105800
Model Family:                Binomial   Df Model:                           14
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -45535.
Date:                Sun, 06 Aug 2023   Deviance:                       91071.
Time:                        09:44:19   Pearson chi2:                 1.06e+05
No. Iterations:                     5   Pseudo R-squ. (CS):            0.07964
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------

## Model Evaluation

#### **Assess Model Fit 2**

In [34]:
# Create VM dataset
vm_df_train_7_select_features = vm.init_dataset(
    dataset=df_train_7_drop_features,
    target_column=target_column)
vm_df_test_7_select_features = vm.init_dataset(
    dataset=df_test_7_drop_features,
    target_column=target_column)

# Create VM model
vm_model_fit_2_final = vm.init_model(
    model = model_fit_2_final, 
    train_ds=vm_df_train_7_select_features, 
    test_ds=vm_df_test_7_select_features)

2023-08-06 09:44:19,738 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:44:19,744 - INFO(validmind.vm_models.dataset): Inferring dataset types...
2023-08-06 09:44:21,149 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2023-08-06 09:44:21,150 - INFO(validmind.vm_models.dataset): Inferring dataset types...


In [35]:
from validmind.tests.model_validation.statsmodels.RegressionCoeffsPlot import RegressionCoeffsPlot

test_context_models_fit_2_final = TestContext(models = [vm_model_fit_2_final])

metric = RegressionCoeffsPlot(test_context_models_fit_2_final)
metric.run()
await metric.result.log()
metric.result.show()

In [36]:
from validmind.tests.model_validation.statsmodels.RegressionModelsCoeffs import RegressionModelsCoeffs

metric = RegressionModelsCoeffs(test_context_models_fit_2_final)
metric.run()
await metric.result.log()
metric.result.show()

In [37]:
from validmind.tests.model_validation.statsmodels.LogRegressionConfusionMatrix import LogRegressionConfusionMatrix

test_context_model_fit_2_final = TestContext(model= vm_model_fit_2_final)

# Configure test parameters
params = {
    "cut_off_threshold": 0.5,
}

metric = LogRegressionConfusionMatrix(test_context_model_fit_2_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [38]:
from validmind.tests.model_validation.statsmodels.RegressionROCCurve import RegressionROCCurve

metric = RegressionROCCurve(test_context_model_fit_2_final)
metric.run()
await metric.result.log()
metric.result.show()

In [39]:
from validmind.tests.model_validation.statsmodels.GINITable import GINITable

metric = GINITable(test_context_model_fit_2_final)
metric.run()
await metric.result.log() 
metric.result.show()

In [40]:
from validmind.tests.model_validation.statsmodels.LogisticRegPredictionHistogram import LogisticRegPredictionHistogram

# Configure test parameters
params = {
    "title": "Histogram of Probability of Default",
}

metric = LogisticRegPredictionHistogram(test_context_model_fit_2_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [41]:
from validmind.tests.model_validation.statsmodels.LogisticRegCumulativeProb import LogisticRegCumulativeProb

# Configure test parameters
params = {
    "title": "Cumulative Probability of Default",
}

metric = LogisticRegCumulativeProb(test_context_model_fit_2_final, params)
metric.run()
await metric.result.log()
metric.result.show()

In [42]:
from validmind.tests.model_validation.statsmodels.ScorecardHistogram import ScorecardHistogram

# Configure test parameters
params = {
    "target_score": 600,
    "target_odds": 50,
    "pdo": 20,
    "title": "Histogram of Credit Scores",
}

metric = ScorecardHistogram(test_context_model_fit_2_final, params)
metric.run()
await metric.result.log()
metric.result.show()